In [1]:
import os
import json
from langchain.llms.bedrock import Bedrock

#get the model ids from here
#https://docs.aws.amazon.com/bedrock/latest/userguide/model-ids.html

# Bedrock boto3

In [1]:

import boto3
import json

bedrock = boto3.client(service_name='bedrock-runtime', region_name='us-east-1')

modelId = 'anthropic.claude-v2'
accept = 'application/json'
contentType = 'application/json'
body = json.dumps({
    "prompt": "Human: Tell me about the English Premier League. Assistant:",
    "max_tokens_to_sample": 500
})

response = bedrock.invoke_model(body=body, modelId=modelId, accept=accept, contentType=contentType)
response_body = json.loads(response.get('body').read())

print(response_body.get('completion'))

 Here are some key facts about the English Premier League (EPL):

- The EPL is the top level of the English football league system. It was formed in 1992 when clubs broke away from the Football League to take advantage of lucrative TV rights deals.

- There are 20 clubs in the Premier League. During the course of a season, each club plays 38 matches against the other clubs (home and away). 

- At the end of the season, the club at the top of the table is crowned champion. The top teams also qualify for the UEFA Champions League. The bottom three teams are relegated to the lower Championship league.

- Some of the biggest and most successful Premier League clubs include Manchester United, Liverpool, Chelsea, Arsenal, Manchester City and Tottenham Hotspur. 

- The EPL is one of the most popular sports leagues in the world. Matches are broadcast in over 200 countries to billions of viewers. 

- Top players from around the world are drawn to the Premier League given its profile and lucrati

# Bedrock + LangChain

In [5]:
from langchain.llms.bedrock import Bedrock

llm = Bedrock( #create a bedrock llm client
    region_name = 'us-east-1',
    model_id = 'anthropic.claude-v2',
    model_kwargs= {"max_tokens_to_sample": 500}
)
prompt = "Tell me about Real Madrid"
response_text = llm.predict(prompt) #return a response to the prompt
print(response_text)

 Here are some key facts about Real Madrid:

- Real Madrid is a professional football club based in Madrid, Spain. They play in La Liga, the top tier of Spanish football.

- Founded in 1902, Real Madrid is one of the most successful and valuable sports teams in the world. They have won a record 13 UEFA Champions League titles and 34 La Liga titles, among many other trophies.

- Real Madrid plays its home matches at the Santiago Bernabéu Stadium, which has a capacity of over 80,000 spectators. The stadium was built in 1947 and is located in the heart of Madrid.

- Some of the greatest players in football history have played for Real Madrid, such as Alfredo Di Stéfano, Ferenc Puskás, Zinedine Zidane, Ronaldo Nazário, and Cristiano Ronaldo. 

- Current stars of the team include Karim Benzema, Luka Modric, Toni Kroos, and goalkeeper Thibaut Courtois. The team is coached by Carlo Ancelotti.

- Real Madrid has a worldwide fanbase and is estimated to have over 200 million fans. They are one o

In [3]:
prompt = "What is the largest city in Vermont?"
response_text = llm.predict(prompt) #return a response to the prompt
print(response_text)


Vermont does not have any cities. It is the second least populated state in the United States. The largest town in Vermont is


### Inference parameters

In [4]:
def get_inference_parameters(model): #return a default set of parameters based on the model's provider
    bedrock_model_provider = model.split('.')[0] #grab the model provider from the first part of the model id
    
    if (bedrock_model_provider == 'anthropic'): #Anthropic model
        return { #anthropic
            "max_tokens_to_sample": 512,
            "temperature": 0, 
            "top_k": 250, 
            "top_p": 1, 
            "stop_sequences": ["\n\nHuman:"] 
           }
    
    elif (bedrock_model_provider == 'ai21'): #AI21
        return { #AI21
            "maxTokens": 512, 
            "temperature": 0, 
            "topP": 0.5, 
            "stopSequences": [], 
            "countPenalty": {"scale": 0 }, 
            "presencePenalty": {"scale": 0 }, 
            "frequencyPenalty": {"scale": 0 } 
           }
    
    elif (bedrock_model_provider == 'cohere'): #COHERE
        return {
            "max_tokens": 512,
            "temperature": 0,
            "p": 0.01,
            "k": 0,
            "stop_sequences": [],
            "return_likelihoods": "NONE"
        }
    
    elif (bedrock_model_provider == 'meta'): #META
        return {
            "temperature": 0,
            "top_p": 0.9,
            "max_gen_len": 512
        }

    else: #Amazon
        #For the LangChain Bedrock implementation, these parameters will be added to the 
        #textGenerationConfig item that LangChain creates for us
        return { 
            "maxTokenCount": 512, 
            "stopSequences": [], 
            "temperature": 0, 
            "topP": 0.9 
        }

In [5]:
#Build the function to call Bedrock with the appropriate inference parameters for the model.
def get_text_response(model, input_content): #text-to-text client function
    
    model_kwargs = get_inference_parameters(model) #get the default parameters based on the selected model
    
    llm = Bedrock( #create a Bedrock llm client
        credentials_profile_name=os.environ.get("BWB_PROFILE_NAME"), #sets the profile name to use for AWS credentials (if not the default)
        region_name='us-east-1', #sets the region name (if not the default)
        endpoint_url=os.environ.get("BWB_ENDPOINT_URL"), #sets the endpoint URL (if necessary)
        model_id=model, #use the requested model
        model_kwargs = model_kwargs
    )
    
    return llm.predict(input_content) #return a response to the prompt

In [6]:
prompt = "What is the largest city in Vermont?"
response_text = get_text_response('anthropic.claude-v2',prompt)
print(response_text)

 The largest city in Vermont is Burlington.


### Temperature

In [9]:
#Build the function to call Bedrock with the appropriate inference parameters for the model.
def get_text_response(model, input_content, temperature): #text-to-text client function
    model_kwargs = get_inference_parameters(model) #get the default parameters based on the selected model
    model_kwargs['temperature'] = temperature if 'temperature' in model_kwargs else model_kwargs.get('temperature')

    llm = Bedrock( #create a Bedrock llm client
        region_name='us-east-1', #sets the region name (if not the default)
        model_id=model, #use the requested model
        model_kwargs = model_kwargs
    )
    
    return llm.predict(input_content) #return a response to the prompt

In [12]:
prompt = "Write a haiku about a long journey:"
response_text = get_text_response('ai21.j2-ultra-v1',prompt, 0.5)
print(response_text)


Leaving home
Heading out on a journey
Destination unknown

Leaving home
Heading out on a journey
Destination unknown


# Prompting

In [7]:
from langchain.prompts import (
    ChatPromptTemplate,
    FewShotChatMessagePromptTemplate,
)

In [24]:
examples = [
    {"input": "Messi", "output": "Barcelona"},
    {"input": "Figo", "output": "Real Madrid"},
]
example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}"),
        ("ai", "{output}"),
    ]
)
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)
final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You respond with the most famous\
                    team a player belonged to. Just the team name"),
        few_shot_prompt,
        ("human", "{input}"),
    ]
)

print(few_shot_prompt.format())

Human: Messi
AI: Barcelona
Human: Figo
AI: Real Madrid


In [22]:
final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You respond with the most famous\
                    team a player belonged to. Just the team name"),
        few_shot_prompt,
        ("human", "{input}"),
    ]
)

In [61]:
from langchain.llms.bedrock import Bedrock
import json

llm = Bedrock( #create a bedrock llm client
    region_name = 'us-east-1',
    model_id = 'cohere.command-text-v14',
    model_kwargs={"temperature": 0}
)
chain = final_prompt | llm

chain.invoke({"input": "Maradona"})


{'generations': [{'finish_reason': 'COMPLETE', 'id': 'cded2d40-46bd-4fac-bb9e-649f859ad963', 'text': ' Napoli'}], 'id': 'af716a91-3671-4758-8baf-a8de3b4536ab', 'prompt': 'System: You respond with the most famous team a player belonged to. Just the team name\nHuman: Messi\nAI: Barcelona\nHuman: Figo\nAI: Real Madrid\nHuman: Maradona'}


' Napoli'